In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re

# Cargar el modelo y el tokenizer
tokenizer = AutoTokenizer.from_pretrained("ncfrey/ChemGPT-4.7M")
model = AutoModelForCausalLM.from_pretrained("ncfrey/ChemGPT-4.7M")

SPECIAL_TOKENS = {"[CLS]", "[SEP]", "[PAD]", "[UNK]", "[BOS]", "[EOS]", "[MASK]"}

# Función para decodificar tokens a SMILES
def decodificar_tokens(tokens):
    mol = []
    for tok in tokens:
        if tok in SPECIAL_TOKENS:
            continue
        # Quitar corchetes solo de tokens especiales que no sean químicos
        # Por ejemplo, [C] o [O] son válidos y se mantienen
        if tok.startswith("[") and tok.endswith("]"):
            # Solo quitar corchetes si no es un elemento químico válido
            contenido = tok[1:-1]
            # Si el contenido es un símbolo químico o token especial interno como Ring/Branch, mantenerlo
            if re.match(r'^[A-Za-z0-9@=#+\\/-]+$', contenido):
                mol.append(contenido)
            else:
                mol.append(tok)  # mantener tal cual
        else:
            mol.append(tok)
    return "".join(mol)

# Función para generar SMILES
def generar_smiles(input_text, max_length=60):
    # Tokenizar la entrada
    inputs = tokenizer(input_text, return_tensors="pt")

    # Generar secuencia de tokens
    outputs = model.generate(
        inputs['input_ids'],
        max_length=max_length,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=1.0,
        eos_token_id=tokenizer.convert_tokens_to_ids("[EOS]")
    )

    # Convertir los tokens generados a tokens legibles
    tokens = tokenizer.convert_ids_to_tokens(outputs[0])

    # Decodificar los tokens a SMILES
    smiles = decodificar_tokens(tokens)

    return smiles

# Función para postprocesar SMILES
def postprocesar_smiles(tokens_string):
    # Encontrar todos los tokens especiales
    pattern = re.compile(r'\[.*?\]')
    tokens = pattern.split(tokens_string)       # partes de texto fuera de corchetes
    matches = pattern.findall(tokens_string)   # tokens entre corchetes

    result = []
    branch_stack = []
    ring_open = {}  # track de apertura/cierre de anillos

    for i in range(len(tokens)):
        # agregar texto fuera de corchetes
        result.append(tokens[i])

        if i < len(matches):
            tok = matches[i]

            # Branch -> (
            if tok.startswith("[Branch"):
                result.append("(")
                branch_stack.append(")")

            # Ring -> manejar apertura/cierre
            elif tok.startswith("Ring"):
                num = re.findall(r'\d+', tok)
                if num:
                    n = num[0]
                    # si no estaba abierto, abrimos
                    if n not in ring_open:
                        ring_open[n] = True
                    # si estaba abierto, se cierra
                    else:
                        del ring_open[n]
                    result.append(n)

            # Otros tokens (por seguridad)
            else:
                result.append(tok)

    # cerrar todas las ramas abiertas
    while branch_stack:
        result.append(branch_stack.pop())

    return "".join(result)

# Ejemplo de uso
input_text = "C(C(=O)O)N"  # Entrada en formato SMILES
smiles_generado = generar_smiles(input_text)

print("Tokens generados:", smiles_generado)
print("SMILES generado:", postprocesar_smiles(smiles_generado))

ModuleNotFoundError: No module named 'torch'